# Config

In [2]:
# Import dependencies
import pandas as pd
from datetime import datetime
from sleeperpy import User, Leagues


In [53]:
# Settings
date_string = datetime.today().strftime('%Y%m%d')
extra_defensive_points = 18
pr_bonus = 1
kr_bonus = 3
username = 'dirtywizard'

# Extract

In [68]:
# Read database
preds = pd.read_csv('db/records_preds.csv')
preds = preds.loc[preds['date_prediction']==preds['date_prediction'].max()]
mults = pd.read_csv('db/pts_multipliers.csv')
rostLims = pd.read_csv('db/roster_limits.csv')

In [64]:
def processRosters(league_id):
    # Get rosters for all league members
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    # Explode the player columns to find how they are placed on the roster
    result = pd.DataFrame(columns=['owner_id', 'player_id'])
    for col in ['taxi', 'starters', 'players']:
        df = rosters[['owner_id', col]].explode(col)
        df = df.rename(columns={col:'player_id'})
        df[col] = True
        result = result.merge(df, how='outer', on=['owner_id', 'player_id'])
    return result

In [24]:
def procesOwners(league_id):
    owners = pd.DataFrame(Leagues.get_users(league_id))
    list = []
    for i in range(len(owners)):
        if 'team_name' in owners['metadata'][i]:
            list.append(owners['metadata'][i]['team_name'])
        else:
            list.append(owners['display_name'][i])
    owners['franchise_name'] = list
    owners = owners[['user_id', 'franchise_name', 'display_name']]
    return owners

In [25]:
# Get User's Sleeper ID
user_id = User.get_user(username)['user_id']
# Get the User's Sleeper league ID
league_id = Leagues.get_all_leagues(user_id, 'nfl', 2023)[0]['league_id']
# Get all players on rosters
rosters = processRosters(league_id)
# Get info on all owners
owners = procesOwners(league_id)

# Transform

In [69]:
# Customize predictions for league
# multiply by points multiplier
for col in mults.columns[1:]:
    preds[col] = preds[col] * mults.loc[mults['username']==username, col][0]
# Add up point columns
preds['pts_proj'] = preds[mults.columns[1:]].sum(axis=1)
# Add extras for defense starting values
preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] = preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] + extra_defensive_points

# Adjust point calculation based on PR/KR
preds.loc[preds['PR']==True, 'pts_proj'] = preds.loc[preds['PR']==True, 'pts_proj'] + (pr_bonus)
preds.loc[preds['KR']==True, 'pts_proj'] = preds.loc[preds['KR']==True, 'pts_proj'] + (kr_bonus)

# Calculate relative points
preds['pts_proj'] = preds['pts_proj'].fillna(0)
preds = preds.sort_values(by='pts_proj', ascending=False, ignore_index=True)
for pos in rostLims.columns[1:]:
    cond1 = preds['position']==pos
    floor = preds.loc[cond1, 'pts_proj'].reset_index(drop=True)[rostLims.loc[rostLims['username']==username, pos][0]]
    preds.loc[cond1, 'rel_proj'] = preds.loc[cond1, 'pts_proj'] - floor

# Reduce
preds = preds[['week', 'player_id', 'pts_proj', 'rel_proj', 'full_name', 'age', 'years_exp', 'weight', 'height',
       'team', 'position', 'fantasy_positions', 'depth_chart_order',
       'depth_chart_position', 'status', 'injury_status', 'injury_notes',
       'injury_start_date', 'practice_participation', 'practice_description','active'
       ]]

In [70]:
preds.columns

Index(['player_id', 'full_name', 'birth_date', 'age', 'weight', 'height',
       'team', 'position', 'fantasy_positions', 'depth_chart_order',
       'depth_chart_position', 'status', 'injury_status', 'injury_notes',
       'injury_start_date', 'practice_participation', 'practice_description',
       'years_exp', 'active', 'week', 'date_prediction', 'pts_proj',
       'rel_proj'],
      dtype='object')

In [61]:
# Merge
# Merge in points
full = preds \
    .merge(rosters, how='left', on='player_id') \
    .merge(owners, how='left', left_on='owner_id', right_on='user_id')

In [ ]:
# Left off here

In [ ]:
# Sum points across the season
totals = full.groupby('player_id')['pts_proj'].sum().reset_index()
unis = full.drop_duplicates(subset='player_id', ignore_index=True).drop(columns='pts_proj')
sums = unis.merge(totals, how='left', on='player_id')

In [ ]:
# Tidy Up
sums = sums[['player_id', 'franchise_name', 'relpts_proj', 'pts_proj', 'full_name', 'age', 'team', 'position','depth_chart_order', 
       'years_exp', 'weight', 'height','taxi', 'starters', 'players', 'active', 
       'fantasy_positions', 'depth_chart_position', 'PR', 'KR',
       'status', 'injury_status', 'injury_notes','injury_start_date', 'practice_participation', 'practice_description',
       ]]
sums = sums.sort_values(by='relpts_proj', ignore_index=True, ascending=False)
sums.loc[sums['position']=='DEF', 'full_name'] = sums.loc[sums['position']=='DEF', 'team']

# Waiver Report

In [ ]:
# waiver report
minpts = sums.loc[(sums['franchise_name']=='Croccity Body Snatchers')&(sums['taxi'].isna())]['pts_proj'].min()
cond1 = sums['franchise_name']=='Croccity Body Snatchers'
cond2 = sums['franchise_name'].isna()
cond3 = sums['pts_proj']>=minpts
waivers = sums.loc[(cond1|cond2)&cond3].reset_index(drop=True)
waivers.to_csv(f'waivers_{date_string}.csv', index=False)

# Write to CSV

In [ ]:
sums.to_csv(f'processed_{date_string}.csv', index=False)

In [ ]:
# Todo:
# Print any sleeper players who are not already in the ids dataset
# Strip out final R from rookies in sharks
# Build Waiver report
# Build trade report
# RosterOptimizer
# Viz franchise points

In [ ]:
# Starter report
sums.loc[sums['starters']==True].groupby('franchise_name')['relpts_proj'].sum().sort_values()

franchise_name
DangeRUSS Last Ride          86.8844
Comeback 👑’s                173.2645
kevinbash                   188.1524
Idk much about soccer       232.3899
2014champ                   248.5451
Count of Monte Christian    287.5669
GusTheBus                   410.7942
📜 Providence 🪬 Spirits 🥂    451.2059
Verdanks Vacqueros FFC      553.5242
Pretty Big Wieners          610.4773
Saskatoon Squatches         789.8972
Croccity Body Snatchers     860.2586
Name: relpts_proj, dtype: float64

In [ ]:
sums.loc[sums['starters']==True].groupby('franchise_name')['pts_proj'].sum().sort_values()

franchise_name
DangeRUSS Last Ride          160.0080
Comeback 👑’s                1833.2381
kevinbash                   1925.8060
Idk much about soccer       2028.1385
2014champ                   2121.7737
Count of Monte Christian    2160.7955
GusTheBus                   2283.6228
📜 Providence 🪬 Spirits 🥂    2324.0345
Verdanks Vacqueros FFC      2426.5528
Pretty Big Wieners          2483.3059
Saskatoon Squatches         2582.2908
Croccity Body Snatchers     2733.0872
Name: pts_proj, dtype: float64

# Merge ids

In [ ]:
# sharks = sharks.drop_duplicates(subset='id_sharks', ignore_index=True)
# sharks['full_name'] = sharks['id_sharks']

In [ ]:
# ids = pd.read_csv('data_sharks/id_dict.csv', dtype={'player_id':'str', 'id_player_sharks':'str'})
# ids = ids[['player_id', 'id_ourlads', 'full_name']]
# ids

In [ ]:
# allids = ids.merge(sharks, how='outer', on='full_name')
# allids

In [ ]:
# allids.to_csv('id_dict2.csv', index=False)

In [ ]:
# # Calculate stats
# ptsDict = {
# 'username':['dirtywizard'],
# 'Att':[0], 
# 'Comp':[0], 
# 'Pass Yds':[.04], 
# 'Pass TDs':[4],
# 'Int':[-2], 
# 'Sck':[0], 
# 'Rush':[.1], 
# 'Rsh Yds':[.1],
# 'Rsh TDs':[6], 
# 'Fum':[-2], 
# 'Tgt':[0], 
# 'RZ Tgt':[0], 
# 'Rec':[.25], 
# 'Rec Yds':[.1], 
# 'Rec TDs':[6], 
# 'Kick Ret Yds':[.1], 
# 'XPM':[1],
# 'XPA':[0], 
# 'FGM':[3], 
# 'FGA':[0], 
# '10-19 FGM':[0], 
# '20-29 FGM':[0], 
# '30-39 FGM':[.5], 
# '40-49 FGM':[1.5],
# '50+ FGM':[2], 
# 'Miss':[0], 
# 'Yds Allowed':[-.027], 
# 'Pts Agn':[-.29], 
# 'Scks':[1], 
# 'DefTD':[6], 
# 'Safts':[2]
# }
# ptsd = pd.DataFrame.from_dict(ptsDict)
# ptsd.to_csv('db/pts_multipliers.csv', index=False)